In [1]:
import torch
import numpy as np
import sys
sys.path.append('./fcn_opt')
sys.path.append('../')

from ttgo import TTGO
import tt_utils
from test_fcns import sine_nD 
from fcn_plotting_utils import plot_surf, plot_contour

%load_ext autoreload
np.set_printoptions(precision=2)
torch.set_printoptions(precision=2)

%autoreload 2

/idiap/temp/sshetty/miniconda/envs/pyml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-vn_v6jr1 because the default path (/idiap/home/sshetty/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#### Define the cost and the correpsonding pdf function

In [3]:
n=3
pdf, cost =  sine_nD(n=n,alpha=1,device=device) # n>=4

#### Define the domain and the discretization

In [4]:
# Define the domain of the function

L = 2 # [-L,L]^n is the domain of the function

# domain of task params: domain of coefficients a and b in Rosenbrock_4D
# Note: a should in (-sqrt(L), sqrt(L)) 
domain_task = [torch.linspace(-np.sqrt(L),np.sqrt(L),1000).to(device)]+[torch.linspace(50,150,1000).to(device)] 
# domain of decison varibales
domain_decision = [torch.linspace(-L,L,1000).to(device)]*(n)
domain = domain_task+domain_decision

In [5]:
# Find the tt-model corresponding to the pdf
tt_model = tt_utils.cross_approximate(fcn=pdf,  domain=domain, 
                        rmax=200, nswp=20, eps=1e-3, verbose=True, 
                        kickrank=5, device=device)

cross device is cuda
Cross-approximation over a 5D domain containing 1e+15 grid points:
iter: 0  | tt-error: 1.159e+00, test-error:1.195e-01 | time:   1.2515 | largest rank:   1
iter: 1  | tt-error: 1.199e-01, test-error:6.690e-09 | time:   1.2934 | largest rank:   6
iter: 2  | tt-error: 0.000e+00, test-error:9.511e-14 | time:   1.3603 | largest rank:  11 <- converged: eps < 0.001
Did 1002000 function evaluations, which took 0.01277s (7.846e+07 evals/s)



### Fit the TT-Model

In [6]:
ttgo = TTGO(tt_model=tt_model,domain=domain,cost=cost,device=device)

#### Specify task parameters

In [7]:
x_task = torch.tensor([1]*3).view(1,-1).to(device) #given task-parameters

### Sample from TT-Model

In [8]:
n_samples_tt = 50

samples = ttgo.sample_tt(n_samples=n_samples_tt, x_task=x_task, alpha=0.9) 


### Choose the best sample as an estimate for optima (initial guess)

In [9]:
best_estimate = ttgo.choose_best_sample(samples)[0]
top_k_estimate = ttgo.choose_top_k_sample(samples,k=10)[0] # for multiple solutions

##### Fine-tune the estimate using gradient-based optimization

In [10]:
ttgo_optimized,_ = ttgo.optimize(best_estimate)

ttgo_optimized_k = 1*top_k_estimate
for i, x in enumerate(ttgo_optimized_k):
    ttgo_optimized_k[i],_ = ttgo.optimize(x)


In [11]:
print("PDF at the estimated point(initial guess): ", pdf(best_estimate))
print("PDF at the TTGO Optima: ", pdf(ttgo_optimized))

PDF at the estimated point(initial guess):  tensor([0.74], device='cuda:0')
PDF at the TTGO Optima:  tensor([0.98], device='cuda:0')


In [12]:
print("Estimated Optima: ", best_estimate)
print("Optima from TTGO: ", ttgo_optimized)

Estimated Optima:  tensor([[1.00, 1.00, 1.00, 2.00, 1.33]], device='cuda:0')
Optima from TTGO:  tensor([[ 1.01, 50.37,  1.01,  2.00,  1.34]], device='cuda:0')
